In [30]:
import sys
sys.path.append('/home/peng/git/Machine_learning_for_reliability_analysis/Preprocess')
sys.path.append('/home/peng/git/Machine_learning_for_reliability_analysis/DAlgorithms')

import Preprocessdata
import RFclass

from __future__ import print_function
from __future__ import division

import logging
from sklearn.datasets import make_classification
from sklearn.cross_validation import cross_val_score

from sklearn.ensemble import RandomForestClassifier as RFC


from bayes_opt import BayesianOptimization
from pandas.core.frame import DataFrame
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.metrics.classification import accuracy_score, confusion_matrix, classification_report
from scipy.interpolate import spline
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score

from sklearn.gaussian_process import GaussianProcess
import timeit

import seaborn as sns
from IPython.core.pylabtools import figsize
from scipy.interpolate import spline
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV
import re
from astropy.io.fits.header import Header
from matplotlib.pyplot import xlim

%matplotlib inline

In [31]:
# define the method that will be used including the keras classifier and label inverse transform for 
#model evaluation

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding


def keras_classifier():

    hidden_size = 2

    model_1 = Sequential()
    model_1.add(Dense(hidden_size, input_dim=encoded_dim, activation='relu'))
#    model_1.add(Dense(hidden_size, activation='relu'))
    model_1.add(Dense(1
                  , activation='sigmoid'))

    model_1.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model_1

def convert_label(b_array):
    
    
    label_predict = []
    length = np.shape(b_array)[0]
    dim = np.shape(b_array)[1]
    
    outputtest = b_array
    
    if dim ==1:
        for i in xrange(0, length):
            if (outputtest[i]>=0.5):
                label_predict.append(1)
            else:
                label_predict.append(0)
        
        
        
    
    
    elif dim == 3:
        
        for i in xrange(0, length):
            if (outputtest[i][0]>=outputtest[i][1]) & (outputtest[i][0]>=outputtest[i][2]):
                
                label_predict.append(0)
                
            elif (outputtest[i][1]>=outputtest[i][0])& (outputtest[i][1]>=outputtest[i][2]):
                label_predict.append(1)
                
            elif (outputtest[i][2]>=outputtest[i][1])& (outputtest[i][2]>=outputtest[i][0]):
                label_predict.append(2)
                
     
               
     
    elif dim ==2:
        for i in xrange(0, length):
            
            
            if (outputtest[i][0]>=outputtest[i][1]):
                
                label_predict.append(0)
            elif (outputtest[i][1]>=outputtest[i][0]):
                label_predict.append(1)
        
        
        

                
    return label_predict  


In [32]:
# Load data set and target values

start = timeit.default_timer()
p= Preprocessdata.standardprocess()


save_path = '/home/peng/Documents/Project_C/Turbofan/Dataset/'
#names = xrange(0,28)

df = pd.read_csv(save_path + 'train_FD004_2_p20_.txt', header=0)

rad_stat = 2
tree_seed = 'rf'

# drop the first column
df = df.drop(df.columns[[0]], axis=1)


#split the dataset into train and label first

df_array = np.array(df)
target_all = df_array[:,-1]
train_all = df_array[:, 0:-1]

# change the label to be binary Optional
#from sklearn import preprocessing
#lb = preprocessing.LabelBinarizer(sparse_output = True)
#target_all_binary = lb.fit_transform(target_all)



#print (train_all[:,0:2])
from sklearn.preprocessing import scale

train_all_scale = scale(train_all)
#print (train_all_scale[:,0:2])

from sklearn.cross_validation import train_test_split
train, test, trainlabel,  testlabel = train_test_split( train_all_scale, target_all, test_size=0.3, random_state=42)

#print (df)

In [33]:
# Variational autoencoder

'''This script demonstrates how to build a variational autoencoder with Keras.
Reference: "Auto-Encoding Variational Bayes" https://arxiv.org/abs/1312.6114
'''
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives
import timeit


start = timeit.default_timer()

#batch_size = 100
original_dim = 24
latent_dim = 2
intermediate_dim = 24
nb_epoch = 5

x = Input(shape=(24,))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(latent_dim, ), mean=0.)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)


def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * objectives.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

vae = Model(x, x_decoded_mean)
vae.compile(optimizer='rmsprop', loss=vae_loss)

vae.fit(train, train,
        shuffle=True,
        nb_epoch=nb_epoch,
        #batch_size=batch_size,
        verbose = 0)

# build a model to project inputs on the latent space
encoder = Model(x, z_mean)

# display a 2D plot of the digit classes in the latent space

train_encoded = encoder.predict(train)
#print(np.shape(train)[1])
test_encoded = encoder.predict(test)
#print(np.shape(test))

encoded_dim = np.shape(train_encoded)[1]

stop = timeit.default_timer()

print ("The running takes %r min" %((stop-start)/60))

The running takes 0.05920158624649048 min


In [5]:
### Sparse autoencoder

from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
# this is the size of our encoded representations
encoding_dim = 6  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_dim= Input(shape=(24,))
# "encoded" is the encoded representation of the input
encoded = Dense(48, activation='tanh')(input_dim)
encoded = Dense(48, activation='tanh', activity_regularizer=regularizers.activity_l1(10e-5))(encoded)

decoded = Dense(24, activation='tanh')(encoded)


# this model maps an input to its reconstruction
autoencoder = Model(input=input_dim, output=decoded)

encoder = Model(input=input_dim, output=encoded)
autoencoder.compile(optimizer='sgd', loss='mse')

start = timeit.default_timer()
autoencoder.fit(train, train,
                nb_epoch=5,
                #batch_size=100,
                shuffle=True
                )

stop = timeit.default_timer()

print ("The running takes %r min" %((stop-start)/60))

# get the encoded representations 
train_encoded = encoder.predict(train)
#print(np.shape(train)[1])
test_encoded = encoder.predict(test)
#print(np.shape(test))

encoded_dim = np.shape(train_encoded)[1]

Epoch 1/5
15226/15226 [==============================] - 0s - loss: 0.3202     
Epoch 2/5
15226/15226 [==============================] - 0s - loss: 0.2027     
Epoch 3/5
15226/15226 [==============================] - 0s - loss: 0.1846     
Epoch 4/5
15226/15226 [==============================] - 0s - loss: 0.1753     
Epoch 5/5
15226/15226 [==============================] - 0s - loss: 0.1698     
The running takes 0.08571791648864746 min


/home/peng/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:514: UserWarning: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 1 is not part of the computational graph needed to compute the outputs: keras_learning_phase.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.
  **kwargs)


In [9]:
### Denosing autoencoder

from keras.layers import Input, Dense
from keras.models import Model

noise_factor = 0.5
train_noisy = train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train.shape) 


train_noisy = np.clip(train_noisy, 0., 1.)


# this is the size of our encoded representations
encoding_dim = 6  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_dim= Input(shape=(24,))
# "encoded" is the encoded representation of the input
encoded = Dense(48, activation='tanh')(input_dim)
encoded = Dense(48, activation='tanh', activity_regularizer=regularizers.activity_l1(10e-5))(encoded)

decoded = Dense(24, activation='tanh')(encoded)


# this model maps an input to its reconstruction
autoencoder = Model(input=input_dim, output=decoded)

encoder = Model(input=input_dim, output=encoded)
autoencoder.compile(optimizer='sgd', loss='mse')

start = timeit.default_timer()
autoencoder.fit(train, train,
                nb_epoch=5,
                #batch_size=100,
                shuffle=True
                )

stop = timeit.default_timer()

print ("The running takes %r min" %((stop-start)/60))

# get the encoded representations 
train_encoded = encoder.predict(train)
#print(np.shape(train)[1])
test_encoded = encoder.predict(test)
#print(np.shape(test))

encoded_dim = np.shape(train_encoded)[1]

Epoch 1/5
15226/15226 [==============================] - 0s - loss: 0.3134     
Epoch 2/5
15226/15226 [==============================] - 0s - loss: 0.2020     
Epoch 3/5
15226/15226 [==============================] - 0s - loss: 0.1863     
Epoch 4/5
15226/15226 [==============================] - 0s - loss: 0.1775     
Epoch 5/5
15226/15226 [==============================] - 0s - loss: 0.1719     
The running takes 0.08430416584014892 min


In [ ]:
### Normal autoencoder

from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 6  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_dim= Input(shape=(24,))
# "encoded" is the encoded representation of the input
encoded = Dense(48, activation='tanh')(input_dim)
encoded = Dense(48, activation='tanh')(encoded)

decoded = Dense(24, activation='tanh')(encoded)


# this model maps an input to its reconstruction
autoencoder = Model(input=input_dim, output=decoded)

encoder = Model(input=input_dim, output=encoded)
autoencoder.compile(optimizer='sgd', loss='mse')

start = timeit.default_timer()
autoencoder.fit(train, train,
                nb_epoch=5,
                #batch_size=100,
                shuffle=True
                )

stop = timeit.default_timer()

print ("The running takes %r min" %((stop-start)/60))

# get the encoded representations 
train_encoded = encoder.predict(train)
#print(np.shape(train)[1])
test_encoded = encoder.predict(test)
#print(np.shape(test))

encoded_dim = np.shape(train_encoded)[1]

In [34]:
## test the kfold in sklearn
start = timeit.default_timer()

from sklearn.cross_validation import KFold
kf = KFold(n=np.shape(train_encoded)[0], n_folds=10, shuffle=False,
                               random_state=42)


model_1 = keras_classifier()

cv_acc=[]
cv_auc= []
cv_prec =[]

test_acc=[]
test_auc= []
test_prec =[]

for train_index, test_index in kf:
  
    model_1.fit(train_encoded[train_index], trainlabel[train_index], nb_epoch=5, verbose = 0)
    
    #model_1.predict(train[test_index], trainlabel[test_index])
    #scores = model_1.evaluate(train_encoded[test_index], trainlabel[test_index], verbose=0)
    #print("%s: %.2f%%" % (model_1.metrics_names[1], scores[1]*100))
    #cv_acc.append(scores[1])
    
    outputtest = model_1.predict(train_encoded[test_index])
#    label_output = convert_label(outputtest)
#    label_test = convert_label(trainlabel[test_index])
    
    label_output = convert_label(outputtest)
    label_test = trainlabel[test_index]
    
    cv_prec.append( precision_score(label_test, label_output))
    cv_acc.append( accuracy_score(label_test, label_output))
    cv_auc.append( roc_auc_score(trainlabel[test_index], outputtest))

    
model_1.fit(train_encoded, trainlabel, nb_epoch = 5, verbose = 0)
outputtest_1 = model_1.predict(test_encoded)
label_output_1 = convert_label(outputtest_1)
label_test_1 = testlabel
test_prec.append( precision_score(label_test_1, label_output_1, average =None)[-1])
test_acc.append( accuracy_score(label_test_1, label_output_1))
test_auc.append( roc_auc_score(testlabel, outputtest_1))

stop = timeit.default_timer()

print ("The running takes %r min" %((stop-start)/60))

The running takes 0.21947216590245563 min


In [37]:
# use tree
classifier = RFC()
             

classifier.fit(train_encoded, trainlabel)
outputtest_2 = classifier.predict(test_encoded)
label_output_2 = convert_label(outputtest_2)
label_test_2 = testlabel

print ( classification_report(label_test_2, label_output_2))




IndexError: tuple index out of range

In [35]:
# VAE results_FD004


print ('cv_accuracy: ', np.mean(cv_acc))

print ('cv_precision: ',np.mean(cv_prec))

print ('cv_auc: ',np.mean(cv_auc))

print ('test_accuracy: ',test_acc)

print ('test_precision: ',test_prec)

print ('test_auc: ',test_auc)

print (confusion_matrix(label_test_1, label_output_1))

print( classification_report(label_test_1, label_output_1))

cv_accuracy:  0.574723724289
cv_precision:  0.735465525591
cv_auc:  0.583236116939
test_accuracy:  [0.60662804795904623]
test_precision:  [0.84377758164165928]
test_auc:  [0.60715018922581632]
[[3547  177]
 [2743  956]]
             precision    recall  f1-score   support

        0.0       0.56      0.95      0.71      3724
        1.0       0.84      0.26      0.40      3699

avg / total       0.70      0.61      0.55      7423



In [10]:
# VAE results_FD003

print ('cv_accuracy: ', np.mean(cv_acc))

print ('cv_precision: ',np.mean(cv_prec))

print ('cv_auc: ',np.mean(cv_auc))

print ('test_accuracy: ',test_acc)

print ('test_precision: ',test_prec)

print ('test_auc: ',test_auc)

print (confusion_matrix(label_test_1, label_output_1))

print( classification_report(label_test_1, label_output_1))

cv_accuracy:  0.956682403433
cv_precision:  0.975845603808
cv_auc:  0.984976123342
test_accuracy:  [0.97931954636424279]
test_precision:  [0.99395567494963066]
test_auc:  [0.99803370223679733]
[[1456    9]
 [  53 1480]]
             precision    recall  f1-score   support

        0.0       0.96      0.99      0.98      1465
        1.0       0.99      0.97      0.98      1533

avg / total       0.98      0.98      0.98      2998



In [7]:
# sparse results

print ('cv_accuracy: ', np.mean(cv_acc))

print ('cv_precision: ',np.mean(cv_prec))

print ('cv_auc: ',np.mean(cv_auc))

print ('test_accuracy: ',test_acc)

print ('test_precision: ',test_prec)

print ('test_auc: ',test_auc)

print (confusion_matrix(label_test_1, label_output_1))

print( classification_report(label_test_1, label_output_1))

cv_accuracy:  0.957577374692
cv_precision:  0.974726325475
cv_auc:  0.992117557123
test_accuracy:  [0.98421697824088261]
test_precision:  [0.99139520590043029]
test_auc:  [0.99885681811243487]
[[3197   28]
 [  75 3226]]
             precision    recall  f1-score   support

        0.0       0.98      0.99      0.98      3225
        1.0       0.99      0.98      0.98      3301

avg / total       0.98      0.98      0.98      6526



In [11]:

# denosing results

print ('cv_accuracy: ', np.mean(cv_acc))

print ('cv_precision: ',np.mean(cv_prec))

print ('cv_auc: ',np.mean(cv_auc))

print ('test_accuracy: ',test_acc)

print ('test_precision: ',test_prec)

print ('test_auc: ',test_auc)

print (confusion_matrix(label_test_1, label_output_1))

print( classification_report(label_test_1, label_output_1))

cv_accuracy:  0.943326074221
cv_precision:  0.966244105592
cv_auc:  0.990564065713
test_accuracy:  [0.9619981612013484]
test_precision:  [0.99130994528484073]
test_auc:  [0.99705562561497696]
[[3198   27]
 [ 221 3080]]
             precision    recall  f1-score   support

        0.0       0.94      0.99      0.96      3225
        1.0       0.99      0.93      0.96      3301

avg / total       0.96      0.96      0.96      6526

